# Task 1 - RAG Model for QA Bot:

File System in VS code:-

Main_Folder:
    |- Document
          |- File.pdf      # The file which content to feed data to your business Bot.
    |- venv                # Enviroment for the model.     
    |- Sub_folder          # The folder in which .ipynb is stored.
          |- ChatBot.ipynb # For business Q/A.
    |- .env                # This variable enviroment where all API keys are stored.          
    |- requirement.txt     # A text file which use to store all uded installation libraries

Used APIs list & others component
    |- OpenAI_API_KEY 
    |- Pinecone_API_KEY
    |- Pinecone_enviroment # To store embended vectors database

This is a reuseable code by the changing only document can use for other task

In [21]:
# Importing libraries 

import pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [23]:
import os

In [ ]:
# Data loading

def read_doc(file_path):
    file_loader = PyPDFLoader(file_path)  # Use PyPDFLoader for single PDF files
    documents = file_loader.load()  # Load the content of the PDF
    return documents


doc = read_doc(r'C:\LangChain\documents\Paper_On_DropOut.pdf')

# Display the content
for page in doc:
    print("Metadata:", page.metadata)  
    print("Content:", page.page_content[:500])  

len(doc)

In [33]:
# Divide the docs into chunks

def chunk_data(docs, chunk_size= 1000, chunk_overlap= 50):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size= chunk_size, chunk_overlap= chunk_overlap)
    doc= text_splitter.split_documents(docs)
    return doc


In [ ]:
documents= chunk_data(docs= doc)
documents

In [ ]:
# Embedding Technique of OpenAI

embeddings= OpenAIEmbeddings(api_key= os.environ['OpenAI_API_KEY'])
embeddings


In [ ]:
# Creates vectors

vectors= embeddings.embed_query("How are you?")
vectors

In [ ]:
# Vector searchDB in Pinecone

pinecone.init(
    api_key= "Pinecone_API_KEY",
    enviroment= "Enviroment_name_from_Pinecone"
)
index_name= "Index_name_from_Pinecone"

In [ ]:
index= Pinecone.from_documents(doc, embeddings, index_name= index_name)

In [38]:
# Consine similarity to retrieve results from vectorDB

def retireve_query(query, k= 2):
    matching_result= index.similarity_search(query, k= k)
    return matching_result

In [40]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm= OpenAI(model_name= "Model_Name", temperature= 0.5)
chain= load_qa_chain(llm, chian_type= "stuff")

In [41]:
# Search answers from VectorDB
def retireve_answer(query):
    doc_search= retireve_query(query)
    print(doc_search)
    response= chain.run(input_documents= doc_search, quetion= query)
    return response 

In [ ]:
user_query= "Who invented Dropout?"
answer= retireve_answer(user_query)
print(answer) 